In [1]:
#import the necessary libraries
import re
import os
import requests
import ipywidgets as wg
import urllib.request, json
from collections import Counter
from pycountry import countries

In [2]:
#Download and save the top 4 most common species in a given country

def countryOfInterest(country):
    """ Returns a list that contains dictionaries that contain links
        to download bird recordings, file names among other details.
    
    Args: 
        country: a string of the name of the country the recordings were done
    """
    recordings = []
    
    """the following three lines take care of countries with more than one name e.g. 'United Kingdom'
        to match the format required for advanced search in Xeno-canto"""
    
    link='https://www.xeno-canto.org/api/2/recordings?query=cnt:" "' 
    link = link.replace(' ', country)
    link = link.replace(' ', '%20')
    with urllib.request.urlopen(link) as url:
        data = json.loads(url.read().decode())
        
    pages = data['numPages'] # get the number of pages available for the query
    
    for page in range(pages): # iterate through the pages
        page  = '&page=' + str(page + 1)
        link='https://www.xeno-canto.org/api/2/recordings?query=cnt:" "' + page #specify the page in the search query
        link = link.replace(' ', country)
        link = link.replace(' ', '%20')
        with urllib.request.urlopen(link) as url:
            content = json.loads(url.read().decode())
            content = content['recordings']
            for dic in content:
                recordings.append(dic)
        
    
    return recordings


def fileSelect(country):
    """Returns a dictionary that comprise of links to download the 
    recordings with the the file names as the keys
    
    Args:
        country: a string of the country the recordings were taken
    
    """
    birds_dict = {}
    extension = '.mp3'
    recordings = countryOfInterest(country)
    birds = [name['en'] for name in recordings]
    num_birds = Counter(birds)
    most_frequent = num_birds.most_common(4)
    
    for bird in most_frequent: #this loop removes unknown birds from the most frequent bird species
        if bird[0] == 'Identity unknown':
            most_frequent = num_birds.most_common(5) #add the fifth most common bird species
            most_frequent.remove(bird) #remove the bird species labeled as unknown
            
    for bird in most_frequent:
        file_plus_links = {}
        for recording in recordings:
            if recording['en'] == bird[0]:
                name = recording['file-name'][:-3]
                name = re.sub(r'[^\w\s]', '', name) #drop punctuations in the file name
                name = name + extension
                
                file_plus_links.update({name:'https:' + recording['file']})
                
        birds_dict.update({bird[0]:file_plus_links})

    return birds_dict


def fileDownload(country):
    """ Downloads and save the audio recordings in the specified path
    Args:
        country: a string of the country the recordings were taken
    """
    birds_dict = fileSelect(country)
    for bird in birds_dict:
        if not os.path.exists(bird):
            os.makedirs(bird)    
        for file in birds_dict[bird]:
            url = birds_dict[bird][file]
            myfile = requests.get(url)
            open(os.path.join(bird, file), 'wb').write(myfile.content)






In [3]:
#Develop a dropdown to select country of choice

countries_list = list(countries) #access countries from the database object from pycountry
drop_down_display = [('Choose Country', -1)]

for indx, i in enumerate(countries_list):
    drop_down_display.append((i.name, indx)) #append tuples containing country names and index
                                            # for dropdown options

def composeCountries(index):
    """Prepare list of countries to be indexed by the Dropdown
        
    """
    list_of_counries = []
    for i in countries_list:
        list_of_counries.append(i.name)

    list_of_counries.append('Null')
    
    if list_of_counries[index] != 'Null':
        print('Downloading...')
        country  = list_of_counries[index]
        fileDownload(country)
        print('Download Complete.')

drop_down = wg.Dropdown(options=drop_down_display,
           description='Country:',
           disabled=False)


wg.interact(composeCountries, index=drop_down)

interactive(children=(Dropdown(description='Country:', options=(('Choose Country', -1), ('Aruba', 0), ('Afghan…

<function __main__.composeCountries(index)>